In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-search:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F67158%2F7460879%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240203%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240203T150219Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dab6374445f72c957fe4604d38d1b8fe5ac7800f977609a4d4cbd41601dc1bbc4f23a00bdefc06f7f46152643bcda05de9b6e125755f786461b528c4e1a19bc8a0383f53894f546833a10d57c1c9023460074c4e8058a2f5b2dff6f8386564ac78b535eeba84d4cb4e97961ca7bda759374df9ada48794745b5e7fc0fe56d8989c00402b23c229054f921d5b85c2d14baf4be16aa3316f9c144f0af9fe65466eba37c5e781968df218f33090fa5d9484c89bf6c594b2956bda9ba8c9f0609533423a6e30746f258937101586ef2d4a04289e7d800bc14f98c61c95588140357252622ce3be0c621e268db6f840711676a95e5044ead215e3f3e01e53744c8e607,arttrain:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4394459%2F7545804%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240203%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240203T150219Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5ca451a133a3e54490ae982d8c2d1303dd3b277c7b7cf1b61d091d8cfff0d852411890a26a29dc2741939af624be3a5e9f46102bf7998673544f7d153c9b09d1671080897fa691a58536c295bacee8aa194d4cdf980095c02054b1b9ba4085519ba550eb84c9d4e03f14650f5959545f32299e70345d1524c9addb59281d10ee1e24efb169ac7477132c5027f8e1480ac0b80d7e9497fe144503ade8bb72a41ba29aea8555bd652d33203b1a243aac87221320a0532c9919321b72dfce48b8ecb00f4487b4be121f8b8d7a7e5ee375a85fec416832c5923b9f1aec752c55d37ee81d21ac99fe6311fc438b30bd2a2df4d4f510e1f70925bfcfe6e73f3aea34c1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Installing CLIP

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
from transformers import AutoProcessor, CLIPModel,CLIPProcessor,AutoModel
import os
import pandas as pd
from PIL import Image
import torch
import glob
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
import numpy as np
from torch import nn, optim
from tqdm.notebook import tqdm
#from .autonotebook import tqdm as notebook_tqdm
import clip
import torchvision.transforms as T
import matplotlib.pyplot as plt
BATCH_SIZE = 8
EPOCH = 5
device = 'cuda'

# Preparing Model and Data

In [ ]:
src_dir = '/kaggle/input/image-search/test/images'
query_dir = '/kaggle/input/image-search/queries/queries'
submission = pd.read_csv('/kaggle/input/image-search/sample_submission.csv')
train_dir = '/kaggle/input/arttrain/train'
#model = CLIPModel.from_pretrained('laion/CLIP-ViT-L-14-DataComp.XL-s13B-b90K').cuda
#processor = AutoProcessor.from_pretrained('laion/CLIP-ViT-L-14-DataComp.XL-s13B-b90K')
model = AutoModel.from_pretrained('laion/CLIP-ViT-L-14-DataComp.XL-s13B-b90K').cuda().eval()
processor = CLIPProcessor.from_pretrained('laion/CLIP-ViT-L-14-DataComp.XL-s13B-b90K')
submission['dot_class'] = 22
submission['cosine_class'] = 22

In [ ]:
train_transform = T.Compose([
    T.Resize(256),
    T.RandomHorizontalFlip(p=0.5),
    T.TrivialAugmentWide(),
    T.CenterCrop(224),
    T.ToTensor()
])

In [ ]:
# List all name folders (assuming "name" is the desired naming convention)
name_folders = [name for name in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, name))]
img = []
name = []
for var in name_folders:
  img2 = glob.glob(train_dir+'/'+var+"/*.jpg")
  img += img2
  name+= [var]*len(img2)
name = [x for x in name if x!='.ipynb_checkpoints']

In [ ]:
df = pd.DataFrame()
df['id'] = name
df['path'] = img
df

In [ ]:
d = {}
for i in set(df['id'].tolist()):
    d[i] = df[df['id'] == i]['path'].tolist()
sel_img = df['path'].tolist()
sel_id = list(set(df['id'].tolist()))

In [ ]:
torch.Tensor(processor(images=[Image.open(sel_img[0])]).pixel_values[0]).shape

In [ ]:
train_img_paths, test_img_paths = train_test_split(sel_id, test_size=0.2, random_state=42)
d_train = {k: d[k] for k in train_img_paths}
d_test = {k: d[k] for k in test_img_paths}
len(d_train), len(d_test)

In [ ]:
transform = T.ToPILImage()
class LogoDataset(Dataset):
    def __init__(self, data, preprocess):
        self.preprocess = preprocess
        self.img_paths = []
        self.captions = []
        for cap, img_path in data.items():
            for img in img_path:
                self.img_paths.append(img)
                self.captions.append(cap)
                self.img_paths.append(img+'tmp')
                self.captions.append(cap)
        self.processed_cache = {}
        for _ ,img_path in data.items():
            for i in range(len(img_path)):
                img = Image.open(sel_img[0])
                imgc = transform(train_transform(img))
                self.processed_cache[img_path[i]] = torch.Tensor(processor(images=[Image.open(sel_img[0])]).pixel_values[0])
                self.processed_cache[img_path[i]+'tmp'] = torch.Tensor(processor(images=[imgc]).pixel_values[0])
        # print(self.img_paths)
        self.path2label = {path: self.img_paths.index(path) for path in self.img_paths}

    def check(self):
        print(self.captions)

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = self.processed_cache[img_path]
        caption = self.captions[idx]
        label = self.path2label[img_path]
        return image, caption, label

In [ ]:
train_dataset = LogoDataset(d_train, processor)
test_dataset = LogoDataset(d_test, processor)
len(train_dataset), len(test_dataset)

# DataLoader

In [ ]:
# https://github.com/pytorch/pytorch/blob/e5742494f6080c8e6f43c37689fc18a7c4b39dfd/torch/utils/data/dataloader.py#L145
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, labels, n_classes, n_samples):
        self.labels = labels
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.n_dataset = len(self.labels)
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < self.n_dataset:
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return self.n_dataset // self.batch_size

train_labels = torch.tensor([item[2] for item in train_dataset])
train_sampler = BalancedBatchSampler(train_labels, BATCH_SIZE, 1)
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler)

test_labels = torch.tensor([item[2] for item in test_dataset])
test_sampler = BalancedBatchSampler(test_labels, BATCH_SIZE, 1)
test_dataloader = DataLoader(test_dataset, batch_sampler=test_sampler)

In [ ]:
for i, item in enumerate(train_sampler):
    labels = []
    for idx in item:
        label = train_dataset[idx][2]
        labels.append(label)
    break
len(labels), len(set(labels))

In [ ]:
for batch in train_dataloader:
    imgs, txts, labels = batch
    print(imgs.shape)
    print(len(txts))
    print(labels)
    print(labels.shape)
    print(torch.unique(labels).shape)
    break

# Train (Fine-Tuning)

In [ ]:
count = 0
for p in model.parameters():
    if count==0:
        print(p)
        break

In [ ]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)

In [ ]:
torch.cuda.is_available()

In [ ]:
processor(text =['this is earth','I screaming'], padding=True, return_tensors="pt")

In [ ]:
model.get_text_features( **processor(text =['this is earth','I screaming'], padding=True, return_tensors="pt").to('cuda'))

In [ ]:
processor(images=[Image.open(sel_img[0])], padding=True, return_tensors="pt")

In [ ]:
best_te_loss = 1e5
best_ep = -1
for epoch in range(EPOCH):
    print(f"running epoch {epoch}, best test loss {best_te_loss} after epoch {best_ep}")
    step = 0
    tr_loss = 0
    model.train()
    pbar = tqdm(train_dataloader, leave=False)
    print(len(pbar))
    for batch in pbar:
        step += 1
        if step == 1:
            print('start')
        if step%10 == 0:
            print(step)
        optimizer.zero_grad()
        images, texts, _ = batch
        images = images.cuda()
        texts = processor(text = texts, padding=True, return_tensors="pt").to('cuda')
        logits_per_image = model.get_image_features(images)
        logits_per_text = model.get_text_features(**texts)
        #logits_per_image, logits_per_text = model(*images, **texts)
        ground_truth = torch.arange(BATCH_SIZE).to(device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        tr_loss += total_loss.item()
        if device == "cpu":
            optimizer.step()
            scheduler.step()
        else:
            #convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            #clip.model.convert_weights(model)
        pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
    tr_loss /= step

    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_dataloader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, _ = batch
            images = images.to(device)
            texts = processor(text = texts, padding=True, return_tensors="pt").to('cuda')
            logits_per_image = model.get_image_features(images)
            logits_per_text = model.get_text_features(**texts)
            ground_truth = torch.arange(BATCH_SIZE).to(device)

            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            te_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        te_loss /= step

    if te_loss < best_te_loss:
        best_te_loss = te_loss
        best_ep = epoch
        torch.save(model.state_dict(), "best_model.pt")
    print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "last_model.pt")

# Search

In [ ]:
model.load_state_dict(torch.load("../input/clipfinetuneweights/best_model.pt"))
NUM_NEG = 127
NUM_TEST = 1000

In [ ]:
%timeit
with torch.no_grad():
    query_images = []
    query_classes = []
    for file in os.listdir(query_dir):
        inputs = processor(images=[Image.open(os.path.join(query_dir, file)).convert('L')], return_tensors='pt').to('cuda')
        outputs = model.get_image_features(inputs.pixel_values).cuda()
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        query_images.append(outputs)
        query_classes.append(int(file[:-4]))
    query_images = torch.cat(query_images)
    for idx, row in submission.iterrows():
        if not pd.isna(row['class']):
            continue
        inputs = processor(images=[Image.open(os.path.join(src_dir, row['img_file'])).convert('L')], return_tensors='pt').to('cuda')
        outputs = model.get_image_features(inputs.pixel_values).cuda()
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        values = outputs @ query_images.T
        if values.softmax(1).max() > .055:
            submission.at[idx, 'dot_class'] = query_classes[values.argmax().tolist()]
        cosine = torch.cosine_similarity(outputs, query_images)
        if cosine.max() > 0.6:
            submission.at[idx, 'cosine_class'] = query_classes[cosine.argmax().tolist()]
    sub = submission[['img_file',]]


In [ ]:
sub['class'] = submission['dot_class']
sub.to_csv('dot_product_0.055_ft-3_try.csv', index=False)
sub['class'] = submission['cosine_class']
sub.to_csv('cosine_similarity_0.6_ft-3_try.csv', index=False)